In [ ]:
# !pip install sqlite3

In [3]:
import pandas as pd

In [61]:
import sqlite3

def create_database(db_file):
    """Создает базу данных SQLite с заданной структурой."""

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Создаем таблицы
    cursor.execute("""
        CREATE TABLE DOCS (
            doc_id INTEGER PRIMARY KEY,
            doc_name TEXT NOT NULL,
            doc_text TEXT NOT NULL,
            gender TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE WORD (
            word_id INTEGER PRIMARY KEY,
            word_position INTEGER NOT NULL,
            word TEXT NOT NULL,
            lemma TEXT NOT NULL,
            tag TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE SENT (
            sent_id INTEGER PRIMARY KEY,
            doc INTEGER NOT NULL,
            sent_position INTEGER NOT NULL,
            sentence_text TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE INVERTED_INDEX (
            word_id INTEGER NOT NULL,
            sent_id INTEGER NOT NULL
        )
    """)

    cursor.execute("CREATE INDEX idx_word_id ON WORD (word_id)") # связываем айди индексами
    cursor.execute("CREATE INDEX idx_sent_id ON SENT (sent_id)")

    conn.commit()
    conn.close()

if __name__ == "__main__":
    db_file = "sem_notes.db" 
    create_database(db_file)
    print(f"База данных '{db_file}' создана успешно!")

База данных 'sem_notes.db' создана успешно!


Общий датафрейм

In [16]:
df = pd.read_csv('nlp_project_corpus_df.csv')
df

,Unnamed: 0,gender,doc_name,doc_text,sentence_text,sent_position,word,lemma,tag,word_position
0,0,f,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,"Из-за того, что смыслы не имеют материального ...",1,из-за,из-за,PR,1
1,1,f,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,"Из-за того, что смыслы не имеют материального ...",1,того,то,SPRO,2
2,2,f,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,"Из-за того, что смыслы не имеют материального ...",1,что,что,SPRO,3
3,3,f,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,"Из-за того, что смыслы не имеют материального ...",1,смыслы,смысл,S,4
4,4,f,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,"Из-за того, что смыслы не имеют материального ...",1,не,не,PART,5
...,...,...,...,...,...,...,...,...,...,...
9906,9906,f,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,значения,значение,S,206
9907,9907,f,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,или,или,CONJ,207
9908,9908,f,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,формы,форма,S,208
9909,9909,f,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,составных,составной,A,209


Датафрейм уникальных документов

In [26]:
name = []

df_docs = df.copy(deep=True)

for i in range(len(df_docs)):
        #doc_id = i
        gender = df_docs.loc[i, 'gender']
        doc_name = df_docs.loc[i, 'doc_name']
        doc_text = df_docs.loc[i, 'doc_text']
        if doc_name not in name:
                name.append(doc_name)
        else:
                df_docs.drop(i, inplace=True)

df_docs.reset_index(inplace=True)

In [ ]:
docs_ids = {}
# словарь название : айди

for i in range(len(df_docs)):
    docs_ids[df_docs.loc[i, 'doc_name']]=i

docs_ids

Датафрейм уникальных предложений

In [43]:
text = []

df_sent = df.copy(deep=True)

for i in range(len(df_sent)):
        #doc_id = i
        sent_text = df_sent.loc[i, 'sentence_text']
        sent_position = df_sent.loc[i, 'sent_position']
        #print(df_sent.loc[i, 'doc_name'])
        doc = int(docs_ids[(df_sent.loc[i, 'doc_name'])])
        df_sent.loc[i, 'doc'] = doc
        if sent_text not in text:
                text.append(sent_text)
        else:
                df_sent.drop(i, inplace=True)

df_sent.reset_index(inplace=True)

Загружаем данные в базу

In [63]:
conn = sqlite3.connect('sem_notes.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in range(len(df)): # загружаем слова
            word_id = i+1
            word = df.loc[i, 'word']
            lemma = df.loc[i, 'lemma']
            tag = df.loc[i, 'tag']
            word_position = int(df.loc[i, 'word_position'])

            cursor.execute(
                """
                INSERT INTO WORD (word_id, word_position, word, lemma, tag) 
                VALUES (?, ?, ?, ?, ?)
                """, 
                (word_id, word_position, word, lemma, tag)
            )

        for i in range(len(df_sent)): # загружаем предложения
            sent_id = i+1
            doc = df_sent.loc[i, 'doc']
            sent_position = int(df_sent.loc[i, 'sent_position'])
            sent_text = df_sent.loc[i, 'sentence_text']            

            cursor.execute(
                """
                INSERT INTO SENT (sent_id, doc, sent_position, sentence_text) 
                VALUES (?, ?, ?, ?)
                """, 
                (sent_id, doc, sent_position, sent_text)
            )

        for i in range(len(df_docs)): # загружаем документы
            doc_id = i+1
            gender = df_docs.loc[i, 'gender']
            doc_name = df_docs.loc[i, 'doc_name']
            doc_text = df_docs.loc[i, 'doc_text']


            cursor.execute(
                """
                INSERT INTO DOCS (doc_id, doc_name, doc_text, gender) 
                VALUES (?, ?, ?, ?)
                """, 
                (doc_id, doc_name, doc_text, gender)
            )

        # Сохраняем изменения
        conn.commit()

        

except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

